Trial p1

In [9]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [8]:
from transformers import ViTForImageClassification, ViTImageProcessor
from PIL import Image
import torch
import gradio as gr  # If you need a UI later

# **Step 1: Upload the Image in Google Colab**
from google.colab import files
uploaded = files.upload()  # Allows direct upload
image_path = list(uploaded.keys())[0]  # Get uploaded file name

# **Step 2: Load the Pre-Trained Deepfake Detection Model**
model_name = "prithivMLmods/Deep-Fake-Detector-Model"  # ViT-based Deepfake Detector
model = ViTForImageClassification.from_pretrained(model_name)
processor = ViTImageProcessor.from_pretrained(model_name)

# **Step 3: Preprocess the Image**
image = Image.open(image_path).convert("RGB")  # Load and convert to RGB
inputs = processor(images=image, return_tensors="pt")  # Convert image for model

# **Step 4: Perform Inference**
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    fake_score = probabilities[0][1].item()  # Assuming index 1 is deepfake class

# **Step 5: Display the Results**
print(f"Deepfake Probability: {fake_score * 100:.2f}%")
if fake_score > 0.4:
    print("❌ The image is likely a deepfake!")
else:
    print("✅ The image appears to be real!")


Saving WhatsApp Image 2022-04-09 at 9.23.18 AM (1).jpeg to WhatsApp Image 2022-04-09 at 9.23.18 AM (1).jpeg
Deepfake Probability: 51.66%
❌ The image is likely a deepfake!


# **Integrated final section**

In [16]:
# Install required dependencies
!pip install -q transformers torch torchvision pillow huggingface_hub tensorflow opencv-python
!pip install -q sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 10.8 MB/s eta 0:00:00


In [19]:

# Import necessary libraries
from transformers import ViTForImageClassification, ViTImageProcessor
from PIL import Image
import torch
import torchvision.transforms as transforms
import cv2
import numpy as np
import os
from google.colab import files
from huggingface_hub import hf_hub_download
from tensorflow.keras.models import load_model

# Function to load multiple models
def load_models():
    model_info = {
        "prithivMLmods/Deep-Fake-Detector-Model": "transformers",
        "Wvolf/ViT_Deepfake_Detection": "transformers",
        "MaanVad3r/DeepFake-Detector": "keras",
        "not-lain/deepfake": "custom"
    }

    models = []
    processors = []

    for model_name, model_type in model_info.items():
        if model_type == "transformers":
            try:
                model = ViTForImageClassification.from_pretrained(model_name)
                processor = ViTImageProcessor.from_pretrained(model_name)
                models.append((model, processor, model_type))
                print(f"✅ Successfully loaded transformers model: {model_name}")
            except Exception as e:
                print(f"❌ Error loading transformers model {model_name}: {e}")
        elif model_type == "keras":
            try:
                model_filename = "cnn_model.h5"
                hf_hub_download(repo_id=model_name, filename=model_filename, local_dir=".")
                model = load_model(model_filename)
                models.append((model, None, model_type))
                print(f"✅ Successfully loaded Keras model: {model_name}")
            except Exception as e:
                print(f"❌ Error loading Keras model {model_name}: {e}")
        elif model_type == "custom":
            try:
                model = torch.hub.load('huggingface/pytorch-transformers', 'model', model_name, trust_remote_code=True)
                processor = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', model_name, trust_remote_code=True)
                models.append((model, processor, model_type))
                print(f"✅ Successfully loaded custom model: {model_name}")
            except Exception as e:
                print(f"❌ Error loading custom model {model_name}: {e}")

    return models

# Function to preprocess images
def preprocess_image(image, processor=None, model_type="transformers"):
    if model_type == "transformers" and processor:
        inputs = processor(images=image, return_tensors="pt")
        return inputs
    elif model_type == "keras":
        image = image.resize((128, 128))
        img_array = np.array(image) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        return img_array
    elif model_type == "custom" and processor:
        inputs = processor(images=image, return_tensors="pt")
        return inputs
    else:
        raise ValueError("Unsupported model type or missing processor.")

# Function to predict deepfake probability for an image
def predict_image(image, models):
    fake_scores = []

    for model, processor, model_type in models:
        inputs = preprocess_image(image, processor, model_type)

        if model_type == "transformers":
            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits
                probabilities = torch.nn.functional.softmax(logits, dim=-1)
                fake_score = probabilities[0][1].item()
                fake_scores.append(fake_score)
        elif model_type == "keras":
            predictions = model.predict(inputs)
            fake_score = predictions[0][0]
            fake_scores.append(fake_score)
        elif model_type == "custom":
            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits
                probabilities = torch.nn.functional.softmax(logits, dim=-1)
                fake_score = probabilities[0][1].item()
                fake_scores.append(fake_score)

    if fake_scores:
        avg_fake_score = np.mean(fake_scores)
        return avg_fake_score
    else:
        return None

# Function to process video frames and extract deepfake probability
def process_video(video_path, models, frame_skip=30):
    cap = cv2.VideoCapture(video_path)
    frame_scores = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if frame_count % frame_skip == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(frame_rgb)
            frame_score = predict_image(image, models)
            if frame_score is not None:
                frame_scores.append(frame_score)
        frame_count += 1

    cap.release()

    if frame_scores:
        avg_video_score = np.mean(frame_scores)
        return avg_video_score
    else:
        return None

# Main function to handle uploads and predictions
def main():
    # Load models
    models = load_models()
    if not models:
        print("No models loaded. Exiting.")
        return

    # Upload file(s) from Google Colab
    print("Please upload an image or video file...")
    uploaded = files.upload()

    for file_name in uploaded.keys():
        print(f"\nProcessing file: {file_name}")
        ext = os.path.splitext(file_name)[1].lower()

        if ext in [".jpg", ".jpeg", ".png"]:
            try:
                image = Image.open(file_name).convert("RGB")
                avg_fake_score = predict_image(image, models)
                if avg_fake_score is not None:
                    print(f"🔍 **Final Deepfake Probability:** {avg_fake_score * 100:.2f}%")
                    if avg_fake_score > 0.35:
                        print("❌ **The image is likely a deepfake!**")
                    else:
                        print("✅ **The image appears to be real!**")
                else:
                    print("⚠️ No valid predictions were made.")
            except Exception as e:
                print(f"Error processing image {file_name}: {e}")

        elif ext == ".mp4":
            try:
                avg_video_score = process_video(file_name, models)
                if avg_video_score is not None:
                    print(f"🔍 **Final Deepfake Probability (Video):** {avg_video_score * 100:.2f}%")
                    if avg_video_score > 0.55:
                        print("❌ **The video is likely a deepfake!**")
                    else:
                        print("✅ **The video appears to be real!**")
                else:
                    print("⚠️ No valid predictions were made.")
            except Exception as e:
                print(f"Error processing video {file_name}: {e}")

        else:
            print("⚠️ Unsupported file type. Please upload an image or an MP4 video.")

# Run the program
if __name__ == "__main__":
    main()


✅ Successfully loaded transformers model: prithivMLmods/Deep-Fake-Detector-Model
✅ Successfully loaded transformers model: Wvolf/ViT_Deepfake_Detection


✅ Successfully loaded Keras model: MaanVad3r/DeepFake-Detector
❌ Error loading custom model not-lain/deepfake: Unrecognized configuration class <class 'transformers_modules.not-lain.deepfake.a5b43e66d4fcae4475108f7462b179021d733bfc.deepfakeconfig.DeepFakeConfig'> for this kind of AutoModel: AutoModel.
Model type should be one of AlbertConfig, AlignConfig, AltCLIPConfig, AriaConfig, AriaTextConfig, ASTConfig, AutoformerConfig, BambaConfig, BarkConfig, BartConfig, BeitConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BitConfig, BlenderbotConfig, BlenderbotSmallConfig, BlipConfig, Blip2Config, BloomConfig, BridgeTowerConfig, BrosConfig, CamembertConfig, CanineConfig, ChameleonConfig, ChineseCLIPConfig, ChineseCLIPVisionConfig, ClapConfig, CLIPConfig, CLIPTextConfig, CLIPVisionConfig, CLIPSegConfig, ClvpConfig, LlamaConfig, CodeGenConfig, CohereConfig, Cohere2Config, ConditionalDetrConfig, ConvBertConfig, ConvNextConfig, ConvNextV2Config, CpmAntCo

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


Saving ajiyrjfyzp-0.mp4 to ajiyrjfyzp-0.mp4

Processing file: ajiyrjfyzp-0.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
🔍 **Final Deepfake Probability (Video):** 51.76%
❌ **The video is likely a deepfake!**


# **Error correction**# New Section